In [4]:
!pip install gym

In [6]:
!pip install stable-baselines

In [8]:
!pip install tensorflow==1.14.0

  Using cached https://files.pythonhosted.org/packages/f4/28/96efba1a516cdacc2e2d6d081f699c001d414cc8ca3250e6d59ae657eb2b/tensorflow-1.14.0-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/91/2d/2ed263449a078cd9c8a9ba50ebd50123adf1f8cfbea1492f9084169b89d9/tensorboard-1.14.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/3c/d5/21860a5b11caf0678fbc8319341b0ae21a07156911132e0e71bffed0510d/tensorflow_estimator-1.14.0-py2.py3-none-any.whl
  Found existing installation: tensorboard 2.0.2
    Uninstalling tensorboard-2.0.2:
      Successfully uninstalled tensorboard-2.0.2
  Found existing installation: tensorflow-estimator 2.0.1
    Uninstalling tensorflow-estimator-2.0.1:
      Successfully uninstalled tensorflow-estimator-2.0.1
  Found existing installation: tensorflow 2.0.0
    Uninstalling tensorflow-2.0.0:
      Successfully uninstalled tensorflow-2.0.0


In [12]:
!gym

/bin/sh: 1: gym: not found


In [11]:
import pandas as pd
import numpy as np
import gym
from stable_baselines import A2C,DQN
# from stable_baselines.common.policies import MlpPolicy
# from stable_baselines.common.vec_env import DummyVecEnv

# from dmarket.environments import SingleAgentTrainingEnv
# from dmarket.agents import UniformRandomAgent, GymRLAgent
# from dmarket.info_settings import OfferInformationSetting
# from dmarket.engine import MarketEngine

ModuleNotFoundError: No module named 'tensorflow.python.estimator'

Let's create a training environment for baselines with 5 fixed buyers and sellers and 1 RL agent to be trained

In [ ]:
fixed_agents = [
    UniformRandomAgent('seller', 90),
    UniformRandomAgent('seller', 90),
    UniformRandomAgent('seller', 90),
    UniformRandomAgent('seller', 90),
    UniformRandomAgent('seller', 90),
    UniformRandomAgent('buyer', 110),
    UniformRandomAgent('buyer', 110),
    UniformRandomAgent('buyer', 110),
    UniformRandomAgent('buyer', 110),
    UniformRandomAgent('buyer', 110),
]

rl_agent = GymRLAgent('buyer', 110, discretization=20)
setting = OfferInformationSetting(5)

def get_env(rl_agent, fixed_agents, setting):
    return SingleAgentTrainingEnv(rl_agent, fixed_agents, setting)

env = DummyVecEnv([lambda: get_env(rl_agent, fixed_agents, setting)]) # wrap it for baselines

Initialize a deep-Q RL model with MLP policy and high learning rate:

In [ ]:
model = DQN("MlpPolicy", env, verbose=1, learning_rate=0.05)

In [ ]:
%%time
model.learn(total_timesteps=10000)

Let's put this agent to the test

In [ ]:
rl_agent.model = model

In [ ]:
def get_reward(agent, deals):
    if not agent.name in deals:
        return 0

    deal_price = deals[agent.name]
    if agent.role == 'buyer':
        return agent.reservation_price - deal_price
    else:
        return deal_price - agent.reservation_price

def play_games(agents, setting, n_games=100, max_steps=30):
    buyer_ids =  [
        agent.name
        for agent in agents
        if agent.role == 'buyer'
    ]
    seller_ids =  [
        agent.name
        for agent in agents
        if agent.role == 'seller'
    ]
    ids = set(buyer_ids + seller_ids)
    market = MarketEngine(buyer_ids, seller_ids, max_steps=max_steps)
    
    rewards = pd.DataFrame(0, index=np.arange(n_games), columns=ids)
    for game_idx in range(n_games):
        while market.done != ids:
            observations = setting.get_states(ids, market)
            unmatched_agents = [
                agent for agent in agents
                if agent.name not in market.done
            ]
            offers = {
                agent.name: agent.get_offer(observations[agent.name])
                for agent in unmatched_agents
            }
            deals = market.step(offers)
            for agent in unmatched_agents:
                rewards[agent.name][game_idx] = get_reward(agent, deals)
        market.reset()
    return rewards

In this simple situation, the agent can get decent rewards:

In [ ]:
play_games(fixed_agents + [rl_agent], setting, 10)

In [ ]:
play_games(fixed_agents + [rl_agent], setting, 100).describe()

Let's see if it can act as a seller as well:

In [ ]:
rl_seller = GymRLAgent('seller', 90, model=model, name='myseller')

In [ ]:
play_games(fixed_agents + [rl_seller], setting, 100).describe()

We see it performs slightly worse.